In [1]:
# importations 
import pandas as pd

### Data pre-processing

In [2]:
df = pd.read_csv("00. input/train.csv", sep=';', decimal=".")
df_test = pd.read_csv("00. input/test.csv", sep=';', decimal=".")

In [3]:
y = df["Machine failure"]
X = df.drop(columns=["Machine failure"]).select_dtypes(include='float64')
X_test = df_test.select_dtypes(include='float64')

### Je tente une simple regression logistique

In [4]:
# simple pipeline

from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)), # remplissage des valeurs manquantes par un imputer
    ('scaler', StandardScaler()), # mise à l'échelle des données
    ('classifier', LogisticRegression())
])

scores = cross_val_score(pipeline, X, y, cv=5, scoring='roc_auc')
print('Scores de validation croisée :', scores)
print('Précisiono moyenne :', scores.mean())

Scores de validation croisée : [0.78453717 0.80810091 0.81442028 0.81125144 0.80949138]
Précisiono moyenne : 0.8055602345007975


In [5]:
# je le fais à la main pour voir à quoi ressemblent la méthode predict_proba

from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)), # remplissage des valeurs manquantes par un imputer
    ('scaler', StandardScaler()), # mise à l'échelle des données
    ('classifier', LogisticRegression())
])

# on fit le modèle
pipeline.fit(X, y)

# on prédit les probas
aux = pd.DataFrame(pipeline.predict_proba(X_test), columns=["proba_0", "proba_1"])

### un essai de soumission (soumission_1.csv)

In [6]:
# essai de soumission en utilisant la pipeline basique

from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)), # remplissage des valeurs manquantes par un imputer
    ('scaler', StandardScaler()), # mise à l'échelle des données
    ('classifier', LogisticRegression())
])

# je fit avec tte la donnée de train
pipeline.fit(X, y)

# je prédis la proba de la machine failure donc la 2 emme colonne 
y_test = pd.concat([df_test["id"], pd.DataFrame(pipeline.predict_proba(X_test), columns=["proba_0", "Machine failure"])["Machine failure"]], axis=1)

In [7]:
# y_test.to_csv("01. output/soumission_1.csv", decimal='.', sep=',', index=False)